# Movie Recomender

In this notebook we are going to explore several ways of recommending movies. The first Method is using "collabrative filtering." In it's essence we are grouping movies as similar by how similarly people have rated them. For instance if "User A" and "User B" rated "Movie X"  5/5 and 2/5 respectivly and rated "Movie Y"  4/5 and 1/5 respectivly then "Movie X" and "Movie Y" are considered similar and either one will be a recomendation for the other.

## Cleaning the Data

The data comes in the form of three tables: 
* Ratings: Has the users and how they rated various movies
* Movies_Metadata: Has all data concerning the movie such as release date and genre
* Credits: Has the people involved in the movie

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
ratings_df = pd.read_csv('Data/ratings_small.csv')

In [3]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
movies_df = pd.read_csv('Data/movies_metadata.csv')

In [5]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
#Dropping columns that are not related to the rating and wont serve as filters 
movies_df.drop(columns=[
            'belongs_to_collection',
            'homepage','imdb_id', 
            'original_title', 
            'overview', 
            'spoken_languages', 
            'status', 'tagline', 
            'poster_path',
            'video'
        ], inplace = True)

In [7]:
'''
The table contains columns that have information as JSON objects however appear as strings. 
To be able to use them as JSON objects we use literal_eval which will take the string and interret as a JSON object
In the below code we are taking the primary production company and country
'''
from ast import literal_eval

production_countries = []
production_companies = []
for index, row in movies_df.iterrows():
    # we are using a try and except statment as some of the information is missing and when we hit a keyerror we would like 
    # the column to reflect the missing information with a Null
    try:
        production_countries.append(literal_eval(movies_df['production_countries'][index])[0]['name'])
    except:
        production_countries.append(None)
    try:
        production_companies.append(literal_eval(movies_df['production_companies'][index])[0]['name'])
    except:
        production_companies.append(None) 
movies_df['production_countries'] = production_countries
movies_df['production_companies'] = production_companies
movies_df.head()

,adult,budget,genres,id,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,title,vote_average,vote_count
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,21.9469,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,17.0155,TriStar Pictures,United States of America,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,11.7129,Warner Bros.,United States of America,1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,3.85949,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,8.38752,Sandollar Productions,United States of America,1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0


In [8]:
# we want users to be able to filter by year as there is a correlation between movies made in the same period. 
# Hoever specific date does not matter and therefore we are simplifying th column
movies_df['release_date'] = [str(release_date).split('-')[0] for release_date in movies_df['release_date']]

In [9]:
# the genres come with IDs which are more or less meaningless for our purpose and therefore cleaning up the column so it is a list of genres
movie_genres = []
for genres in movies_df['genres']:
    movie_genres.append([genre['name'] for genre in literal_eval(genres)])
movies_df['genres'] = movie_genres

In [10]:
credits_df = pd.read_csv('Data/credits.csv')
credits_df.head()

,cast,crew,id,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 1686,Unnamed: 1687,Unnamed: 1688,Unnamed: 1689,Unnamed: 1690,Unnamed: 1691,Unnamed: 1692,Unnamed: 1693,Unnamed: 1694,Unnamed: 1695
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# to simplify the filtering we are having it so users will only be filtering by the main actor or their gender
main_actor = []
main_actor_gender = []

for cast in credits_df['cast']:
    try:
        main_actor.append(literal_eval(cast)[0]['name'])
        main_actor_gender.append(literal_eval(cast)[0]['gender'])
    except:
        main_actor.append(None)
        main_actor_gender.append(None)
credits_df['main_actor'] = main_actor
credits_df['main_actor_gender'] = main_actor_gender

In [12]:
# to simplify the filtering we are having it so users will only be filtering by the directo or their gender
# in the list of cast the director tends to be the first crew member
director = []
director_gender = []

for crew in credits_df['crew']:
    try:
        director.append(literal_eval(crew)[0]['name'])
        director_gender.append(literal_eval(crew)[0]['gender'])
    except:
        director.append(None)
        director_gender.append(None)
credits_df['director'] = director
credits_df['director_gender'] = director_gender

In [13]:
movies_df.head()

,adult,budget,genres,id,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,title,vote_average,vote_count
0,False,30000000,"[Animation, Comedy, Family]",862,en,21.9469,Pixar Animation Studios,United States of America,1995,373554033.0,81.0,Toy Story,7.7,5415.0
1,False,65000000,"[Adventure, Fantasy, Family]",8844,en,17.0155,TriStar Pictures,United States of America,1995,262797249.0,104.0,Jumanji,6.9,2413.0
2,False,0,"[Romance, Comedy]",15602,en,11.7129,Warner Bros.,United States of America,1995,0.0,101.0,Grumpier Old Men,6.5,92.0
3,False,16000000,"[Comedy, Drama, Romance]",31357,en,3.85949,Twentieth Century Fox Film Corporation,United States of America,1995,81452156.0,127.0,Waiting to Exhale,6.1,34.0
4,False,0,[Comedy],11862,en,8.38752,Sandollar Productions,United States of America,1995,76578911.0,106.0,Father of the Bride Part II,5.7,173.0


In [14]:
credits_df.drop(columns=['cast', 'crew'], inplace = True)

In [15]:
credits_df.head()

,id,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 1690,Unnamed: 1691,Unnamed: 1692,Unnamed: 1693,Unnamed: 1694,Unnamed: 1695,main_actor,main_actor_gender,director,director_gender
0,862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Tom Hanks,2.0,John Lasseter,2.0
1,8844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Robin Williams,2.0,Larry J. Franco,2.0
2,15602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Walter Matthau,2.0,Howard Deutch,2.0
3,31357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Whitney Houston,1.0,Forest Whitaker,2.0
4,11862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Steve Martin,2.0,Alan Silvestri,2.0


In [16]:
ratings_df.drop(columns=['timestamp'], inplace = True) # remove timestamp of when the rating happened

In [18]:
ratings_df.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


## Merging the tables

all three tables have MovieID as a common key, this appears as MovieID in rating table and ID in the movie_metadata and credits table
we join the movie_metadata table to the rating table. The movie ID which is unique in the other two tables appears multiple times in the rating table
this is obviously due to multiple users who rate a given movie. Therefore to not lose any information during the Join we join everythin to Ratings

In [20]:
# we found that the column was not being read as int64 and some of the numbers in the 4000 range were being interpreted as date
# the movies with oddly formated IDs will be dropped during the join
movies_df.id = pd.to_numeric(movies_df['id'], errors='coerce') 
df_combined = pd.merge(ratings_df, movies_df, left_on='movieId', right_on='id', how='left')

In [21]:
df_combined.head()

,userId,movieId,rating,adult,budget,genres,id,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,title,vote_average,vote_count
0,1,31,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1029,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1061,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1129,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1172,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_combined.drop(columns=['id'], inplace = True) # drop the extra ID column after joining Movie_metadata to ratings

In [21]:
df_combined = pd.merge(df_combined, credits_df, left_on='movieId', right_on='id', how='left') #join credit tableto combines rating and metadeta table

In [22]:
df_combined.drop(columns=['id'], inplace = True) # drop extra ID column after join

In [23]:
df_combined.dropna(inplace = True) #to prevent any errors from user filters we drop NAs

We want to get rid of obscure movies in our database as we feel that this will make the recomendations better

In [24]:
df_combined  = df_combined[df_combined['vote_count'] > 100] 
df_combined.head()

,userId,movieId,rating,adult,budget,genres,original_language,popularity,production_companies,production_countries,...,Unnamed: 1690,Unnamed: 1691,Unnamed: 1692,Unnamed: 1693,Unnamed: 1694,Unnamed: 1695,main_actor,main_actor_gender,director,director_gender


## Making the rating matrix

This matrix will have the user ID as the index and the various movies as columns and the ratings as the values. We take advantage of the pivot_table method from panadas to do this in one step.

In [25]:
import numpy as np
movie_rating_matrix = pd.pivot_table(df_combined, values='rating', index='userId',
                                    columns=['title'], aggfunc=np.sum)
movie_rating_matrix.head()

userId


## The Recommender function

The recommender function is taking a movie inputted by the user and seeing which movie had similar ratings per user to determine correlation.

As an example let's take the movie Cast Away and say User 1 gave it a 5 and User 2 gave it a 1. Cast Away would be highly correlated with 2001 Space Odessy which had User 1 rate it as 5 and User 2 rate it as 2. Cast Away would be poorly correlated with Transformers which had User 1 rate it as 2 and USer 2 rate it as 5.

After gathering a list of similarly rated movies we then join that list with our merged dataframe. At this point we no longer care about individuals ratings since we have the correlations so to prevent duplicates in our recommendations we remove duplicates keeping the first instance of the movie.

We can then apply any additional filtering that the recommnder user might like to add.

In [26]:
df_combined.set_index('title', inplace = True) # this will help use when joining the recommendationts which will have title as index.

In [27]:
def movieRecomender(movie_title, **kwargs):
    '''
    Takes a movie and filters and returns up to 5 recommendations.
    
    Parameters
    ----------
    
    movie_title: str
                 representing movie title.
    
    Keyword Arguments
    ----------
    
    adult: Str 'False' or 'True'
    
    original_language: str
                    2 character code
    
    production_companies: str
    
    production_countries: str
    
    main_actor: str
    
    main_actor_gender: int 
                    1 for female and 2 for male
    director: str
    
    director_gender: int
                    1 for female and 2 for male
    
    date_range: tuple
                (start year, end year)
    
    Returns
    ----------
    
    List of up to 5 movie recommendations
    '''
    if not movie_title in df_combined.index: # return error if movie is not in database
        return "Movie not in database"
    movie_corr = pd.DataFrame(movie_rating_matrix.corrwith(movie_rating_matrix[movie_title]), columns=['Correlation'])
    movie_corr = movie_corr.join(df_combined).sort_values(by='Correlation', ascending=False)
    movie_corr = movie_corr[~movie_corr.index.duplicated(keep='first')]
    movie_corr = movie_corr[movie_corr.index != movie_title]
    for key, value in kwargs.items():
        if key == 'genre': # Since the genres are in a list the filtering is different from other keywords
            movie_corr = movie_corr[movie_corr['genres'].apply(lambda x: value in x)]
        elif key == 'date_range':
            if not type(value) is tuple: # To ensure that the date range is given in tuple
                return "You need to pass in a tuple for range argument"
            movie_corr = movie_corr[movie_corr['release_date'].apply(lambda x: value[0] <= int(x) <= value[1])]
        else:
            try:
                movie_corr = movie_corr[movie_corr[key] == value]
            except KeyError:
                return "This is not a possible filter"
    recommendations =  movie_corr.head()
    return recommendations.index.values.tolist()

## Example output

In [28]:
movieRecomender('The Dark Knight',production_companies = 'Columbia Pictures')

'Movie not in database'

In [29]:
movieRecomender('The Dark Knight',date_range = (1990, 2001))

'Movie not in database'

In [30]:
movieRecomender('The Dark Knight',adult = 'False')

'Movie not in database'

In [31]:
movieRecomender('The Dark Knight',production_companies = 'Columbia Pictures',date_range = (1990, 2001),adult = 'False')

'Movie not in database'

## Testing the errors

1. Inputting non existent movie

In [32]:
movieRecomender('BootCamp Horrors')

'Movie not in database'

2. Not using a Tuple

In [33]:
movieRecomender('The Dark Knight',date_range = 1990)

'Movie not in database'

3. Non-existent keyword argument

In [34]:
movieRecomender('The Dark Knight',is_this_good = True)

'Movie not in database'

## SVD: Singular Vector Decomposition

In [35]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [36]:
reader = Reader()
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

In [37]:
algo = SVD()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9003  0.8976  0.8963  0.8951  0.9004  0.8979  0.0021  
MAE (testset)     0.6908  0.6928  0.6887  0.6884  0.6949  0.6911  0.0025  
Fit time          4.17    4.12    4.14    4.13    4.14    4.14    0.02    
Test time         0.80    0.14    0.14    0.09    0.16    0.27    0.27    


{'test_rmse': array([0.90030281, 0.8975576 , 0.89632728, 0.89512155, 0.9004312 ]),
 'test_mae': array([0.690843  , 0.69282418, 0.68868454, 0.68843709, 0.69486511]),
 'fit_time': (4.170894622802734,
  4.123934507369995,
  4.139643907546997,
  4.128990173339844,
  4.139638900756836),
 'test_time': (0.7966880798339844,
  0.14061999320983887,
  0.14060187339782715,
  0.09372878074645996,
  0.1562185287475586)}

In [38]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [39]:
trainset, testset = train_test_split(data, test_size=.25)

In [40]:
algo.fit(trainset)
predictions = algo.test(testset)

In [41]:
accuracy.rmse(predictions)

RMSE: 0.8983


0.8982969880676491

In [42]:
example = [(1,31,2.5)]

predictions = algo.test(example)
print (predictions)

[Prediction(uid=1, iid=31, r_ui=2.5, est=2.4308959632723712, details={'was_impossible': False})]


#### The next session contains an ALS based approach to predict preferences by userid. The code references medium articles and work on kaggle below. "Diff. Approaches of Building Recommender System" on KAGGLE

#### Fundamental algorithm is NMF. Non-Negative Matrix Factorization (NMF). Find two non-negative matrices (W, H) whose product approximates the non- negative matrix X. This factorization can be used for example for dimensionality reduction, source separation or topic extraction.

In [ ]:
## Read csv file and pivot necessary columns
import numpy as np

ratings = pd.read_csv('./Data_Amar/ratings_small.csv')

nb_users  = ratings_df['userId'].nunique()
nb_movies = ratings_df['movieId'].nunique()

ratings_matrix = ratings_df.pivot_table(index=['userId'],columns=['movieId'],values='rating').reset_index(drop=True)
ratings_matrix.fillna(0, inplace = True)

data_matrix = np.array(ratings_matrix)

print(data_matrix.shape)
print(data_matrix)

In [ ]:
# NMF factorization into user vector and item vector
from sklearn.decomposition import NMF
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import numpy as np
from numpy.linalg import solve
import pandas as pd 

model = NMF(n_components=2, init='random', random_state=42)
user_vec = model.fit_transform(data_matrix)
item_vec = model.components_.T

def implicit_ALS(ratings, user_vec, item_vec, lambda_val, iteration, typ):                 
    
    ctr = 1

    if typ == 'user':
        while ctr <= iteration:
            YTY = item_vec.T.dot(item_vec)
            lambdaI = np.eye(YTY.shape[0]) * lambda_val

            for u in range(user_vec.shape[0]):
                user_vec[u, :] = solve((YTY + lambdaI), 
                                        ratings[u, :].dot(item_vec))
            ctr += 1

        return user_vec
    
    if typ == 'item':
        while ctr <= iteration:
            XTX = user_vec.T.dot(user_vec)
            lambdaI = np.eye(XTX.shape[0]) * lambda_val
            
            for i in range(item_vec.shape[0]):
                item_vec[i, :] = solve((XTX + lambdaI), 
                                        ratings[:, i].T.dot(user_vec))
            ctr += 1
        return item_vec
        
    
user_vec = implicit_ALS(data_matrix, user_vec, item_vec, lambda_val=0.2,
                        iteration=20, typ='user')
item_vec = implicit_ALS(data_matrix, user_vec, item_vec, lambda_val=0.2,
                        iteration=20, typ='item')

def predict_all():
        """ Predict ratings for every user and item. """
        predictions = np.zeros((user_vec.shape[0], 
                                item_vec.shape[0]))
        for u in range(user_vec.shape[0]):
            for i in range(item_vec.shape[0]):
                predictions[u, i] = predict(u, i)   
        return predictions
    
def predict(u, i):
    """ Single user and item prediction. """
    return user_vec[u, :].dot(item_vec[i, :].T)

predict = predict_all()
data_matrix_pred_ALS = np.where(data_matrix == 0, predict, data_matrix)


## For measuring the performance we have to use the predicted matrix
X_pred = np.zeros((nb_users, nb_movies))
for i in range(predict.shape[0]):
    for j in range(predict.shape[1]):
        X_pred[i, j] = (predict[i, j])
        
## The section below generates the RMSE
ind_matrix = np.zeros((nb_users, nb_movies)) #generates a shell matrix
ind_matrix = np.where(data_matrix != 0, 1, 0) #populates with 1 if movie is rated; zero otherwise
x = np.multiply(X_pred, ind_matrix)
RMSE_ALS = np.sqrt(np.mean((x - data_matrix)**2))
print("RMSE of ALS Model is %f." %RMSE_ALS)

## Generate input by asking for Customer ID
a = int(input("Customer ID"))
print("Generating Preferences for customer ID",a)

## Create an indicator matrix to ensure the movie was not rated previously
ind_matrix = np.zeros((nb_users, nb_movies))
ind_matrix = np.where(data_matrix == 0, 1, 0)

## Multiply predicted rating matrix with this indicator matrix to consider only the predicted ones
pred = np.multiply(data_matrix_pred_ALS, ind_matrix)
pred = pred[a-1, :]

## Create a new dataframe which accepts movieID, Rating and a flag (to indicate if move has been rated prior)
## Created a new column called final rating 
column_names = ["movieID", "Rating",'flag','Fin Rating']
pred_df = pd.DataFrame(columns = column_names)

pred_df['flag']=ind_matrix[a-1,:]
pred_df['Rating']=pred
pred_df['movieID']=ratings['movieId']
pred_df['Fin Rating']=pred_df['flag']*pred_df['Rating']
pred_df.nlargest(5, 'Fin Rating')

In [ ]:
# Creating final table
pred_df['movieID']=pred_df['movieID'].apply(str)
movies_id_title=movies_df[['id', 'title']] 
movies_id_title.rename(columns={"id": "movieID"})
pd.merge(pred_df, movies_id_title, left_on='movieID', right_on='id', how='left')